In [126]:
import pandas as pd
import numpy as np
import common as cm

This exercise consists of 3 parts. Finish the first part to get a mark of 3.0. The first two parts to get 4.0. Finish all exercies to get 5.0.

# Part 1: Concordance

Given are the following modes of transport: bus, bike, car, train. Each mode is characterized by 2 cost-type criteria: price and time; and 2 gain-type criteria: comfort and reliability. 

Mode of transport | Time | Comfort | Price | Reliability
--- | --- | --- | --- | ---
 **bus**  | 6 | 3  | 6 | 2
 **bike** | 8 | 2  | 2 | 8
 **car**  | 2 | 10 | 9 | 7
 **train**| 3 | 5  | 5 | 6
 **by foot**| 10 | 2  | 0 | 5

In [127]:
data = {
    'mode': ['bus', 'bike', 'car', 'train', 'foot'],
    'time': [6, 8, 2, 3, 10],
    'comfort': [3, 2, 10, 5, 2],
    'price': [6, 2, 9, 5, 0],
    'reliability': [2, 8, 7, 6, 5]
}

criteria = ['time', 'comfort', 'price', 'reliability']

data = pd.DataFrame(data, columns=['mode', 'time', 'comfort', 'price', 'reliability'])
data

,mode,time,comfort,price,reliability
0,bus,6,3,6,2
1,bike,8,2,2,8
2,car,2,10,9,7
3,train,3,5,5,6
4,foot,10,2,0,5


1.2) Below are the parameters, i.e., threholds, criterion-type, and weights, for each criterion, 

In [128]:
parameters = {'time': {'weight': 4, 'q': 1.0, 'p': 2, 'v': 4, 'type': 'cost'},
 'comfort': {'weight': 2, 'q': 2.0, 'p': 3, 'v': 6, 'type': 'gain'},
 'price': {'weight': 3, 'q': 1.0, 'p': 3, 'v': 5, 'type': 'cost'},
 'reliability': {'weight': 1, 'q': 1.5, 'p': 3, 'v': 5, 'type': 'gain'}}

sum_weights = 10.

pd.DataFrame(parameters, columns=['time', 'comfort', 'price','reliability']).reindex(['type', 'q', 'p', 'v', 'weight']).T

,type,q,p,v,weight
time,cost,1,2,4,4
comfort,gain,2,3,6,2
price,cost,1,3,5,3
reliability,gain,1.5,3,5,1


1.3) Finish the below function for calculating a marginal concordance for $i$-th criterion (gain type) $c_i(g_i(A),g_i(B))$ based on q and p tresholds. 

In [129]:
def getConcordanceCost(gA, gB, q, p):
    return getConcordanceGain(gB, gA, q, p)

def getConcordanceGain(gA, gB, q, p):
    #TODO
    if gB >= gA + p:
        return 0
    if gB <= gA + q:
        return 1
    return 1-(gB - (gA+q))/(p-q)
print(getConcordanceGain(10,5,2,5))
print(getConcordanceCost(10,6.5,2,5))

1
0.5


1.4)  Calculate  comprehensive concordance  index  for  all  criteria  of  alternatives A and B. Remember that comprehensive concordance must be divided by the sum of weights (normalization).  

In [130]:
def getComprehensiveConcordance(A, B, criteria, parameters):
    concordance = 0.0
    for criterium in criteria:
        if parameters[criterium]['type'] == 'gain':
            concordance += getConcordanceGain(A[criterium],B[criterium],parameters[criterium]['q'],parameters[criterium]['p'])*parameters[criterium]['weight']
        else:
            concordance += getConcordanceCost(A[criterium],B[criterium],parameters[criterium]['q'],parameters[criterium]['p'])*parameters[criterium]['weight']
    
    return concordance / sum_weights

1.5) Check comprehensive concordance indexes for C(bus, some transportation):

In [131]:
for alternative_id, alternative_row in data.iterrows():
    print("C({0},{1}) = ".format(0, alternative_id), getComprehensiveConcordance(data.loc[0], alternative_row, criteria, parameters))

C(0,0) =  1.0
C(0,1) =  0.6
C(0,2) =  0.3
C(0,3) =  0.5
C(0,4) =  0.6


1.6) Finish the below function for generating a concordance matrix. Use a majority_threshold as a cutting-level. For hich pairs a concordance is fulfilled?

In [132]:
### TODO
def getConcordanceMatrix(data, criteria, parameters, majority_treshold=0.7):
    concordance_matrix = np.zeros((len(data),len(data)))
    for A_idx, A_row in data.iterrows():
        for B_idx, B_row in data.iterrows():
            if A_idx == B_idx:
                continue
            concordance_matrix[A_idx][B_idx]=1 if getComprehensiveConcordance(A_row,B_row,criteria,parameters) >= majority_treshold else 0
    return concordance_matrix

In [133]:
print(getConcordanceMatrix(data, criteria, parameters))

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [1. 1. 0. 1. 1.]
 [1. 0. 1. 0. 1.]
 [0. 0. 0. 0. 0.]]


# Part 2: outranking graph

2.1) Complete the function for calculating a marginal discordance  $d_i(𝐴,𝐵)$ based on v treshold. $d_i(𝐴,𝐵) = 1$ when A is worse than B on criterion $i$ by at least the veto threshold, zero otherwise.

In [134]:
def getDiscordanceGain(A, B, v):
    if B-v>=A:
        return 1
    return 0

def getDiscordanceCost(A, B, v):
    return getDiscordanceGain(B, A, v)


2.2) Calculate a comprehensive discordance index. $D(a,b) = 1$ if at least one criterion vetoes against aSb.

In [135]:
def getComprehensiveDiscordance(A, B, criteria, parameters):
    for criterium in criteria:
        if parameters[criterium]['type'] == 'gain':
            if getDiscordanceGain(A[criterium],B[criterium],parameters[criterium]['v'])==1:
                return 1
        else:
            if getDiscordanceCost(A[criterium],B[criterium],parameters[criterium]['v'])==1:
                return 1  
    return 0.

2.3) Check comprehensive discordance indexes for D(bus, some transportation):

In [136]:
for alternative_id, alternative_row in data.iterrows():
    print("D({0},{1}) = ".format(0, alternative_id),getComprehensiveDiscordance(data.loc[0], alternative_row, criteria, parameters))


D(0,0) =  0.0
D(0,1) =  1
D(0,2) =  1
D(0,3) =  0.0
D(0,4) =  1


2.4) Finish the below function for calculating a comprehensive discordance matrix.

In [137]:
def getDiscordanceMatrix(data, criteria, parameters):
    discordance_matrix = np.zeros((len(data),len(data)))
    for A_idx, A_row in data.iterrows():
        for B_idx, B_row in data.iterrows():
            if A_idx != B_idx:
                discordance_matrix[A_idx][B_idx] = getComprehensiveDiscordance(A_row, B_row, criteria, parameters)
            
    return discordance_matrix

In [138]:
getDiscordanceMatrix(data, criteria, parameters)

array([[0., 1., 1., 0., 1.],
       [0., 0., 1., 1., 0.],
       [0., 1., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [1., 0., 1., 1., 0.]])

2.5) Now, finish the below function for generating the outranking matrixThis method should take into account both the concordance and discordance matrices.

In [139]:
def getOutrankingMatrix(data, criteria, parameters, majority_treshold):
    concordance_matrix = getConcordanceMatrix(data, criteria, parameters, majority_treshold)
    discordance_matrix = getDiscordanceMatrix(data, criteria, parameters)
    n = len(data)
    outranking_matrix = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            if concordance_matrix[i][j] and not discordance_matrix[i][j]:
                outranking_matrix[i][j] = 1

    return outranking_matrix

In [140]:
outranking_matrix = getOutrankingMatrix(data, criteria, parameters, majority_treshold=0.75)
print(outranking_matrix)

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0.]
 [1. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0.]]


2.6) Change outranking matrix to adjacency list of a following form:

In [141]:
# TODO
def toAdjacencyList(outranking_matrix):
    n = len(outranking_matrix)
    graph = {i:[] for i in range(n)}
    for i in range(n):
        for j in range(n):
            if outranking_matrix[i,j]==1:
                graph[i].append(j)      
    return graph

In [142]:
graph = toAdjacencyList(outranking_matrix)
print(graph)

{0: [], 1: [4], 2: [], 3: [0, 2], 4: []}


2.4) Draw outranking graph, and remove cycles (manually).

In [143]:
cm.PrintGraph(graph, filename="graph_part_2")

2.5) Which mode of transport are in the kernel?

# Part 3: Kernel

3.1) Given is the below outranking matrix

In [144]:
outranking_matrix = np.array(   [[0., 1., 0., 0., 0., 0., 1., 0.],
                                 [0., 0., 1., 0., 1., 0., 0., 0.],
                                 [0., 0., 0., 0., 0., 0., 0., 1.],
                                 [0., 0., 0., 0., 1., 0., 1., 0.],
                                 [0., 0., 0., 0., 0., 1., 0., 0.],
                                 [0., 0., 0., 0., 0., 0., 1., 0.],
                                 [0., 0., 0., 0., 0., 0., 0., 1.],
                                 [0., 0., 0., 0., 0., 0., 0., 0.]]) 

In [145]:
graph = toAdjacencyList(outranking_matrix)

In [146]:
graph

{0: [1, 6], 1: [2, 4], 2: [7], 3: [4, 6], 4: [5], 5: [6], 6: [7], 7: []}

3.2) Display the outranking graph. Which alternatives belong to kernel?

In [147]:
cm.PrintGraph(graph, filename="graph_part_3")

3.3) In this exercise, you are asked to complete the function for constructing a kernel. Firstly, complete the below auxiliary function which reverses edges of the graph.

In [106]:
def getReverseGraph(graph):
    n = len(graph)
    reverse_graph = {i:[] for i in graph}
    for i in graph:
        for j in graph[i]:
            reverse_graph[j].append(i)
    return reverse_graph

3.4) Verify the correctness: compare the below reverse_graph with the above graph variable.

In [107]:
reverse_graph = getReverseGraph(graph)
reverse_graph

{0: [], 1: [0], 2: [1], 3: [], 4: [1, 3], 5: [4], 6: [0, 3, 5], 7: [2, 6]}

3.2) Now, complete the below function for finding a graph kernel. This algorithm should proceed in the following way: <br>
1) Find non-outranked vertices. Add them to the kernel. <br> 
2) Remove the already found vertices from the graph and these vertices which are surpassed by them.<br>
3) Repeat (go to 1) until all vertices are removed from the graph. 

You can use the auxiliary reverse_graph structure. It can be helpfull for finding non-outranked vertices.

In [108]:
def Kernel(graph):
    reverse_graph = getReverseGraph(graph)
    kernel = []
    step=0
    while(len(reverse_graph)>0):
        v = next(iter(reverse_graph))
        e = reverse_graph[v]
        del reverse_graph[v]
        if len(e) ==0:
            kernel.append(v)
            for v_2 in graph[v]:
                for __v in reverse_graph:
                    if v_2 in reverse_graph[__v]:
                        reverse_graph[__v].remove(v_2)
                if v_2 in reverse_graph:
                    del reverse_graph[v_2]
        else:
             reverse_graph[v]=e
    return kernel

In [109]:
Kernel(graph)

[0, 2, 3, 5]

In [123]:
print(outranking_matrix[::,0])
def my_kernel(outranking_matrix):
    length=len(outranking_matrix)
    kernel=[]
    no_kernel=[]
    while not len(kernel)+len(no_kernel) == len(outranking_matrix):
        print(kernel)
        print(no_kernel)
        for column in range(length):
            if column not in kernel and column not in no_kernel and sum(outranking_matrix[:,column])==0:
                kernel.append(column)
                for j in range(length):
                    if j not in kernel and j not in no_kernel and outranking_matrix[column][j]==1:
                        no_kernel.append(j)
                        for k in range(length):
                            outranking_matrix[j][k]=0
    return kernel

print(my_kernel(outranking_matrix))
        

[0. 0. 0. 0. 0. 0. 0. 0.]
[]
[]
[0, 2, 3, 5]
